Import Libraries

In [1]:
import os
import pathlib
import numpy as np
import seaborn as sns
import tensorflow as tf
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout, Resizing, Input, Normalization
from keras.models import Sequential
from IPython import display
import matplotlib.pyplot as plt
%matplotlib widget

We donwload and extract required dataset using `pathlib` module

In [2]:
DATASET_PATH = 'data/'

data_dir = pathlib.Path(DATASET_PATH)

tf.keras.utils.get_file(
    'voicedataset.zip',
    origin='http://aiolearn.com/dl/datasets/voicedata.zip',
    extract=True,
    cache_dir='.',
    cache_subdir='data'
)

print('DONE!')

DONE!


In [3]:
tf.io.gfile.listdir(str(data_dir))

['right',
 'voicedataset.zip',
 'go',
 'no',
 'left',
 'stop',
 'README.md',
 'up',
 'down',
 'yes']

Defining commands

In [4]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))

commands = commands[(commands != 'README.md') & (commands != 'voicedataset.zip')]

print('Available Commands: ', commands )

Available Commands:  ['right' 'go' 'no' 'left' 'stop' 'up' 'down' 'yes']


Creating dataset

In [6]:
X_train, X_test = tf.keras.utils.audio_dataset_from_directory(directory=data_dir,
                                                              batch_size=64,
                                                              validation_split=0.2,
                                                              seed=0,
                                                              output_sequence_length=16000,
                                                              subset='both')

label_names = np.array(X_train.class_names)
print()
print('label : ', label_names)

Found 8000 files belonging to 8 classes.
Using 6400 files for training.
Using 1600 files for validation.

label :  ['down' 'go' 'left' 'no' 'right' 'stop' 'up' 'yes']


Here, we mix and merge the sound and its label with the `squeeze` command.

In [7]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

X_train = X_train.map(squeeze, tf.data.AUTOTUNE)
X_test = X_test.map(squeeze, tf.data.AUTOTUNE)

In [8]:
X_train.element_spec

(TensorSpec(shape=(None, 16000), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))